Brain

In [ ]:
#fitness
#species_id

def initialize():
    raise NotImplementedError

def draw_network():
    raise NotImplementedError

def add_node():
    raise NotImplementedError

def add_connection():
    raise NotImplementedError

def mutate():
    raise NotImplementedError

def load_inputs():
    raise NotImplementedError

def run_network():
    raise NotImplementedError

def get_output():
    raise NotImplementedError

Node

In [ ]:
#node_identifier
#node_type
#node_layer
#sum_input
#sum_output

Connection

In [ ]:
#innovation_id
#in_node_id
#out_node_id
#connection_weight
#enabled
#is_recurrent